In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [3]:
llm.invoke("How will the weather be in chennai today?")

AIMessage(content="I'm sorry, I am not able to provide real-time weather updates. I recommend checking a reliable weather website or app for the most up-to-date information on the weather in Chennai today.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 17, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4f06dbe4-7586-485c-84e9-ba269e0b880b-0', usage_metadata={'input_tokens': 17, 'output_tokens': 39, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
from langchain_core.tools import tool

@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["chennai"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def check_seating_availability(location: str, seating_type: str):
    """Call to check seating availability."""
    if location.lower() == "chennai" and seating_type.lower() == "outdoor":
        return "Yes, we still have seats available outdoors."
    elif location.lower() == "chennai" and seating_type.lower() == "indoor":
        return "Yes, we have indoor seating available."
    else:
        return "Sorry, seating information for this location is unavailable."


tools = [get_weather, check_seating_availability]

"It's 15 degrees Celsius and cloudy."

In [11]:
llm_with_tools = llm.bind_tools(tools)

In [12]:
result = llm_with_tools.invoke("How will the weather be in chennai today?")
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fPQiQ8PxPaQWuYNJoWYL1Kgk', 'function': {'arguments': '{"location":"Chennai"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 84, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2df6ec03-99fa-41f1-8d03-9149e529a39f-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Chennai'}, 'id': 'call_fPQiQ8PxPaQWuYNJoWYL1Kgk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 16, 'total_tokens': 100, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Chennai'},
  'id': 'call_fPQiQ8PxPaQWuYNJoWYL1Kgk',
  'type': 'tool_call'}]

In [14]:
result = llm_with_tools.invoke(
    "How will the weather be in chennai today? Do you still have seats outdoor available?"
)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1t7bfaHi4UInbh17TBtm1mNO', 'function': {'arguments': '{"location": "Chennai"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_DLzOqzxy53QlwMlFgkGqoh2s', 'function': {'arguments': '{"location": "Chennai", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dc936b58-3d85-4695-b589-2e14d9bb95f9-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Chennai'}, 'id': 'call_1t7bfaHi4UInbh17TBtm1mNO', 'type': 'tool_call'}, {'name

In [15]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Chennai'},
  'id': 'call_1t7bfaHi4UInbh17TBtm1mNO',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'Chennai', 'seating_type': 'outdoor'},
  'id': 'call_DLzOqzxy53QlwMlFgkGqoh2s',
  'type': 'tool_call'}]

In [16]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in chennai today? Do you still have seats outdoor available?"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [17]:
messages

[HumanMessage(content='How will the weather be in chennai today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_V6MYhskRnDgJHWwEsSYlQ3wp', 'function': {'arguments': '{"location": "chennai"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_Wfi8a2G4oQ1ZezzXTuIDOsSD', 'function': {'arguments': '{"location": "chennai", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a6d02db4-3e15-4386-a5

In [20]:
tool_mapping = {
    "get_weather": get_weather,
    "check_seating_availability": check_seating_availability,
}

In [18]:
llm_output.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'chennai'},
  'id': 'call_V6MYhskRnDgJHWwEsSYlQ3wp',
  'type': 'tool_call'},
 {'name': 'check_seating_availability',
  'args': {'location': 'chennai', 'seating_type': 'outdoor'},
  'id': 'call_Wfi8a2G4oQ1ZezzXTuIDOsSD',
  'type': 'tool_call'}]

In [21]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [22]:
messages

[HumanMessage(content='How will the weather be in chennai today? Do you still have seats outdoor available?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_V6MYhskRnDgJHWwEsSYlQ3wp', 'function': {'arguments': '{"location": "chennai"}', 'name': 'get_weather'}, 'type': 'function'}, {'id': 'call_Wfi8a2G4oQ1ZezzXTuIDOsSD', 'function': {'arguments': '{"location": "chennai", "seating_type": "outdoor"}', 'name': 'check_seating_availability'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 92, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a6d02db4-3e15-4386-a5

In [23]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in Chennai today is 15 degrees Celsius and cloudy. And yes, we still have seats available outdoors.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 177, 'total_tokens': 202, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-49c77bc8-0f4c-4429-8a93-640819291c40-0', usage_metadata={'input_tokens': 177, 'output_tokens': 25, 'total_tokens': 202, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})